In [1]:
# Import Necessary Modual
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import time
import re
from google.cloud import storage
from google.oauth2 import service_account
import os

In [2]:
# Set credential 
credentials = service_account.Credentials.from_service_account_file('D:/Google Drive/Courtney RA NBA/Keys/nba-prediction-wr-9a502b7df147.json')
# Set credential Path
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="D:/Google Drive/Courtney RA NBA/Keys/nba-prediction-wr-9a502b7df147.json"

In [3]:
# upload file into the GCP browser storage bucket
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    #Uploads a file to the bucket.
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [4]:
# parse final score 
def final_score(ID):
    # keep requesting
    while True:
        pbp_link = 'http://www.espn.com/nba/playbyplay?gameId=' + ID
        pbp_r = requests.get(pbp_link) 
        pbp_soup = BeautifulSoup(pbp_r.content, 'html.parser')
        pbptext = [s.getText() for s in pbp_soup.find_all('td') if s.get('class') != None and ('game-details'in s.get('class'))]
        #when game ends
        if  'End of Game' in pbptext:
            # wait 120 seconds for espn to updati its box score page 
            time.sleep(150)
            # pull scores from box score tab
            stats_link = 'http://www.espn.com/nba/boxscore?gameId=' + ID
            teamorder = pd.read_html(stats_link)[0]
            break
            
    if teamorder.iloc[:,0][0] == 'GS':
        home_score = teamorder.iloc[:,-1].values[0]
        opp_score = teamorder.iloc[:,-1].values[1]
    else:
        home_score = teamorder.iloc[:,-1].values[1]
        opp_score = teamorder.iloc[:,-1].values[0]
            
    # return score in string format '119-120'    
    return str(home_score)+ '-' + str(opp_score)

In [5]:
# pull half time box score 
def new_data_poper(ID):
    while True:
        pbp_link = 'http://www.espn.com/nba/playbyplay?gameId=' + ID
        pbp_r = requests.get(pbp_link) 
        pbp_soup = BeautifulSoup(pbp_r.content, 'html.parser')
        # 
        pbptext = [s.getText() for s in pbp_soup.find_all('td') if s.get('class') != None and ('game-details'in s.get('class'))]
        if 'End of the 2nd Quarter' in pbptext:
            # scrap half stats from box scores
            stats_link = 'http://www.espn.com/nba/boxscore?gameId=' + ID
            teamorder = pd.read_html(stats_link)[0]
            first_team = pd.read_html(stats_link)[1]
            second_team = pd.read_html(stats_link)[2]
            break
        
    if teamorder.iloc[:,0][0] == 'GS':
        home_team = first_team[first_team['Starters'] == 'TEAM']
        opp_team = second_team[second_team['Starters'] == 'TEAM']
    else:
        home_team = second_team[second_team['Starters'] == 'TEAM']
        opp_team = first_team[first_team['Starters'] == 'TEAM']
    
    # transform stats from \d-\d\d format
    for d in [home_team, opp_team]:
        stats_extract(d)
    
    a = home_team[['3PTM','3PTA','FTA','FTM','OREB','DREB','REB','AST','STL','BLK','TO','PF','FGM','FGA']]
    b = opp_team[['3PTM','3PTA','FTA','FTM','OREB','DREB','REB','AST','STL','BLK','TO','PF','FGM','FGA']]
    new_data = pd.concat([a.reset_index(drop=True),b.reset_index(drop=True)],axis=1)
    
    return new_data

In [6]:
# derived columns from original box score table
def stats_extract(df):
    df['3PTA'] = df['3PT'].str.extract('-(\d+)').values
    df['3PTM'] = df['3PT'].str.extract('(\d+)-').values
    df['FTA'] = df['FT'].str.extract('-(\d+)').values
    df['FTM'] = df['FT'].str.extract('(\d+)-').values
    df['FGA'] = df['FG'].str.extract('-(\d+)').values
    df['FGM'] = df['FG'].str.extract('(\d+)-').values

In [7]:
## function to parse stats for history data for a team
def parse_stats(df,side,col_side,col_opp):
    df['threePM'+'_'+side] = np.where(df[col_side].str.contains("makes 3",na=False).values,1,0)
    df['threePA'+'_'+side] = np.where(df[col_side].str.contains("3-pt",na=False).values,1,0)
    df['FTA'+'_'+side] = np.where(df[col_side].str.contains("free",na=False).values,1,0)
    df['FTM'+'_'+side] = np.where(df[col_side].str.contains("(makes technical free)|(makes flagrant free)|(makes free) ",na=False,).values,1,0)
    df['ORB'+'_'+side] = np.where(df[col_side].str.contains("[Oo]ffensive rebound",na=False).values,1,0) - np.where(df[col_side].str.contains("[Oo]ffensive rebound by [Tt]eam",na=False).values,1,0)
    df['DRB'+'_'+side] = np.where(df[col_side].str.contains("[Dd]efensive rebound",na=False).values,1,0) - np.where(df[col_side].str.contains("[Dd]efensive rebound by [Tt]eam",na=False).values,1,0)
    df['TRB'+'_'+side] = df['ORB'+'_'+side].values + df['DRB'+'_'+side].values
    df['AST'+'_'+side] = np.where(df[col_side].str.contains("[Aa]ssist",na=False).values,1,0)
    df['STL'+'_'+side] = np.where(df[col_opp].str.contains("[Ss]teal",na=False).values,1,0)
    df['BLK'+'_'+side] = np.where(df[col_opp].str.contains("[Bb]lock",na=False).values,1,0)
    df['TOV'+'_'+side] = np.where(df[col_side].str.contains("[Tt]urnover",na=False).values,1,0)
    df['PF'+'_'+side] = np.where(df[col_opp].str.contains("([Ss]hooting foul)|([Tt]echnical foul)",na=False).values,1,0) + np.where(df[col_side].str.contains("([Pp]ersonal foul)|([Cc]harge foul)",na=False).values,1,0)
    df['FG'+'_'+side] = np.where(df[col_side].str.contains("makes 2",na=False).values,1,0)  + df['threePM'+'_'+side].values
    df['FGA'+'_'+side] = np.where(df[col_side].str.contains("(misses 2)|(misses 3)",na=False).values,1,0) + df['FG'+'_'+side].values

In [8]:
# main function to parse history data 
def team_parser(team_abb,start_years,end_years):
    
    domain = 'https://www.basketball-reference.com'
    team = '/teams/' + team_abb +'/'
    tail = '_games.html'
    
    # links of box score for each history game in the whole history
    root_links = []
    for year in range(start_years,end_years+1):
        year = year
        root_link = domain + team + str(year) + tail
        root_links.append(root_link)
        
    # schedules for each history game
    request_games = []
    for root_link in root_links: 
        raw_schedule = pd.read_html(root_link, index_col=0)
        if len(raw_schedule) > 1:
            schedule = pd.concat([raw_schedule[0],raw_schedule[1]],axis=0)
        else:
            schedule = raw_schedule[0]
        
        schedule = schedule[schedule.index != 'G']
        schedule.set_index('Date',inplace = True)
    
        game_date = schedule.index
        game_date = game_date.astype(datetime.date)
        
        home_array = np.where(schedule.iloc[:,4].values == '@',0,1)
    
        # request html
        r = requests.get(root_link)
        soup = BeautifulSoup(r.content, 'html.parser')
        a_tags = soup.find_all('a')
        
        # store box score links 
        box_links = []
        for link in a_tags:
            if '/boxscores/' in str(link.get('href')) and ".html" in str(link.get('href')):
                box_links.append(domain + str(link.get('href')).replace('/boxscores/','/boxscores/pbp/'))
        
        #skip "Last game link"
        if box_links[-1] == box_links[0]:
            box_links = box_links[1:]
        
        # for each game
        games = []
        l = len(box_links)
        for i in range(l):
            pbp = pd.read_html(box_links[i], index_col=0)[0]
        
            # index of each quarter
            idx_Q1 = list(pbp.index).index('1st Q')
            idx_Q2 = list(pbp.index).index('2nd Q')
            idx_Q3 = list(pbp.index).index('3rd Q')
            idx_Q4 = list(pbp.index).index('4th Q')
            pbp.reset_index(drop=True,inplace=True)
            if home_array[i] == 1:
                parse_stats(pbp,'team',5,1) # team stat
                parse_stats(pbp,'opp',1,5) #opp stat

            else:
                parse_stats(pbp,'team',1,5) # team stat
                parse_stats(pbp,'opp',5,1) #opp stat

                
            Q1 = pd.DataFrame(pbp.iloc[idx_Q1:idx_Q2, 5:].apply(sum,axis=0)).T
            Q1.columns = ['Q1_'+col for col in Q1.columns]
            Q2 = pd.DataFrame(pbp.iloc[idx_Q2:idx_Q3, 5:].apply(sum,axis=0)).T
            Q2.columns = ['Q2_'+col for col in Q2.columns]
            half = pd.DataFrame(pbp.iloc[idx_Q1:idx_Q3, 5:].apply(sum,axis=0)).T
            half.columns = ['half_'+col for col in half.columns]
            Q3 = pd.DataFrame(pbp.iloc[idx_Q3:idx_Q4, 5:].apply(sum,axis=0)).T
            Q3.columns = ['Q3_'+col for col in Q3.columns]
            first_3Q = pd.DataFrame(pbp.iloc[idx_Q1:idx_Q4, 5:].apply(sum,axis=0)).T
            first_3Q.columns = ['first_3Q_'+col for col in first_3Q.columns]
            Q4 = pd.DataFrame(pbp.iloc[idx_Q4:, 5:].apply(sum,axis=0)).T
            Q4.columns = ['Q4_'+col for col in Q4.columns]
            total = pd.DataFrame(pbp.iloc[:, 5:].apply(sum,axis=0)).T
            total.columns = ['Total_'+col for col in total.columns]
            
            game_stats = pd.concat([Q1,Q2,half,Q3,first_3Q,Q4,total],axis=1)
            games.append(game_stats)
            
        all_games = pd.concat(games)
        all_games['scores_team'] = schedule['Tm'].values
        all_games['scores_opp'] = schedule['Opp'].values
        all_games = all_games.set_index(game_date)
        request_games.append(all_games)
    
    hist_games = pd.concat(request_games)
    
    
    return hist_games

In [9]:
# parse history GSW history data 
d = team_parser('GSW',2015,2018)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


In [10]:
history_half = d[['half_threePM_team','half_threePA_team','half_FTA_team','half_FTM_team','half_ORB_team','half_DRB_team','half_TRB_team','half_AST_team','half_STL_team','half_BLK_team','half_TOV_team','half_PF_team','half_FG_team','half_FGA_team','half_threePM_opp','half_threePA_opp','half_FTA_opp','half_FTM_opp','half_ORB_opp','half_DRB_opp','half_TRB_opp','half_AST_opp','half_STL_opp','half_BLK_opp','half_TOV_opp','half_PF_opp','half_FG_opp','half_FGA_opp']]
history_half = history_half.reset_index(drop=True)
target = list(d['scores_team'].astype(int) - d['scores_opp'].astype(int))

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
from sklearn.linear_model import LassoCV

In [13]:
ls = LassoCV(n_alphas=1000,cv=10)
ls.fit(history_half,target)

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=1000, n_jobs=None, normalize=False,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [14]:
# 2019 schedule
domain = 'https://www.basketball-reference.com'
team = '/teams/' + 'GSW' +'/'
tail = '_games.html'
year = 2019
root_link = domain + team + str(year) + tail

schedule_GSW_2019 = pd.read_html(root_link)[0] 

schedule_GSW_2019 = schedule_GSW_2019[schedule_GSW_2019['G'] != 'G'][['Date','Start (ET)','Unnamed: 5','Opponent']]

schedule_GSW_2019 = schedule_GSW_2019.rename(columns={'Unnamed: 5': 'Home','Start (ET)':'Start Time'})


schedule_GSW_2019['Home'] = np.where(schedule_GSW_2019['Home'] == '@',False,True)
time_split = schedule_GSW_2019['Start Time'].str[:-1].str.split(':')
revised_time = time_split.map(lambda x: str(int(x[0]) + 12)+':' + x[1])

schedule_GSW_2019['Start Date and Time'] = schedule_GSW_2019['Date'].values + ' ' + revised_time

schedule_GSW_2019['Start Date and Time'] = schedule_GSW_2019['Start Date and Time'].map(lambda x: datetime.datetime.strptime(x,'%a, %b %d, %Y %H:%M'))

In [15]:
# get game Id
l = 'http://www.espn.com/nba/team/schedule/_/name/gs'
sch_r = requests.get(l) 
sch_soup = BeautifulSoup(sch_r.content, 'html.parser')

list_gameId = []
for gid in sch_soup.find_all('a'):
    if (gid.get('href') != None) & ('http://www.espn.com/nba/game?gameId=' in str(gid.get('href'))):
        list_gameId.append(str(gid.get('href')))
gameIDs = pd.Series(list_gameId).str.extract('=(\d+)')
gameIDs.columns = ['IDs']

In [16]:
original = pd.DataFrame({'Game Time':schedule_GSW_2019['Start Date and Time'], "Team":'Golden State Warriors', "Opponent": schedule_GSW_2019['Opponent'],'Predicted Final Points Differential':"", 'Final Scores':"" })
# 0
original.iloc[0,-2] = 8
original.iloc[0,-1] = '108-100'
# 1
original.iloc[1,-2] = 10
original.iloc[1,-1] = '124-123'
# 2
original.iloc[2,-2] = 10
original.iloc[2,-1] = '98-100'
# 3
original.iloc[3,-2] = 9
original.iloc[3,-1] = '123-103'
# 4
original.iloc[4,-2] = 10
original.iloc[4,-1] = '144-122'
# 5
original.iloc[5,-2] = 11
original.iloc[5,-1] = '128-100'
# 6
original.iloc[6,-2] = 9
original.iloc[6,-1] = '120-114'
# 7
original.iloc[7,-2] = 12
original.iloc[7,-1] = '149-124'
# 8
original.iloc[8,-2] = 10
original.iloc[8,-1] = '131-121'
# 9
original.iloc[9,-2] = 8
original.iloc[9,-1] = '116-99'
# 10
original.iloc[10,-2] = 7
original.iloc[10,-1] = '117-101'
#11
original.iloc[11,-2] = 10
original.iloc[11,-1] = '111-134'
original.to_html('result.html',index=True)
#12
original.iloc[12,-2] = 8
original.iloc[12,-1] = '116-100'
original.to_html('result.html',index=True)
#13
original.iloc[13,-2] = 11
original.iloc[13,-1] = '116-121'
original.to_html('result.html',index=True)
#14
original.iloc[14,-2] = 10
original.iloc[14,-1] = '110-103'
original.to_html('result.html',index=True)

upload_blob('nbagsw','result.html','result.html')

In [ ]:
# Main Execution!


games = len(original)
for idx in range(15,games):
    while True:
        now = datetime.datetime.now()
        schedule_time = original['Game Time'][idx]
        if (now.year == schedule_time.year) & (now.month == schedule_time.month)  & (now.day == schedule_time.day) & (now.hour == schedule_time.hour) & (now.minute == schedule_time.minute):
            print('match time',schedule_time)
            original.loc[idx,'Predicted Final Points Differential'] = 'Start Predicting!'
            original.loc[idx,'Final Scores'] = 'now playing'
            original.to_html('result.html',index=True)
            upload_blob('nbagsw','result.html','result.html')

            ## activate parser of play by play and aggregate boxscores when hitting half time! store it in new_data
            new_data = new_data_poper(str(gameIDs['IDs'][idx]))
            #predict
            pred = ls.predict(new_data)
            
            ## ouput the prediction to interface
            insert_data =int(pred[0])
            
            original.loc[idx,'Predicted Final Points Differential'] = insert_data

            #upload to bucket
            original.to_html('result.html',index=True)
            upload_blob('nbagsw','result.html','result.html')
            print(original.loc[idx])
            
            ## after game ends paste the game results and data to dataframe
            final = final_score(str(gameIDs['IDs'][idx]))
            original.loc[idx,'Final Scores'] = final
            #upload to bucket
            original.to_html('result.html',index=True)
            upload_blob('nbagsw','result.html','result.html')
            
            #save this game data for future
            append_data = new_data.loc[0]
            append_data.index = history_half.columns
            history_half = history_half.append(append_data,ignore_index=True)
            target_scores = re.split('-',final_score(str(gameIDs['IDs'][idx])))
            append_target = int(target_scores[0]) - int(target_scores[1])
            target.append(append_target)
            
            #retrain model
            ls.fit(history_half,target)
            
            print(idx,'Ends')
            ## exit this while loop
            break
            

match time 2018-11-15 20:00:00


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Game Time                                2018-11-15 20:00:00
Team                                   Golden State Warriors
Opponent                                     Houston Rockets
Predicted Final Points Differential                        5
Final Scores                                     now playing
Name: 15, dtype: object
15 Ends
